In [21]:
from PIL import Image  # install by > python3 -m pip install --upgrade Pillow  # ref. https://pillow.readthedocs.io/en/latest/installation.html#basic-installation
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.backends.backend_pdf
from IPython import display
import reportlab as rl
from reportlab.pdfgen import canvas as rlcanvas
from reportlab.lib.units import cm as ptPerCm

plt.rcParams['figure.figsize'] = [20, 10] # inches

In [3]:
rootPath = "C:\\Users\\dodoa\\Pictures\\MTG\\Proxies\\Test"

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    
pdf_path = os.path.join(rootPath, "result.pdf")
images[0].save(
    pdf_path, "PDF" ,resolution=100.0, save_all=True, append_images=images[1:]
)

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

In [4]:
imagePath = os.path.join(rootPath, "Pictures")
imageList = os.listdir(imagePath)

images = [
    Image.open(os.path.join(imagePath, f))
    for f in imageList
]

img = images[0]

In [22]:
def GetSizeDict(images) :
    widthDict = dict()
    heightDict = dict()
    
    for img in images :
        w,h = img.size
        widthDict[w] = 1 + (widthDict[w] if (w in widthDict.keys()) else 0)
        heightDict[h] = 1 + (heightDict[h] if (h in heightDict.keys()) else 0)
    
    return (widthDict,heightDict)
    # except Error :

In [23]:
def CreateRectangle(bigSize, smallSize, scale = 1) :
    (bigW, bigH) = bigSize
    (smallW, smallH) = smallSize
    
    widthOffset = scale * (bigW - smallW) / 2   
    heightOffset = scale * (bigH - smallH) / 2

    return plt.Rectangle((widthOffset,heightOffset), 
                    scale * smallW,
                    scale * smallH,
                    ls='--',
                    edgecolor='red',
                    facecolor='none',
                    lw=1)
  
def CreateASingleCuttingMark(position, size, color = 'white', lineWidth=3):
    (x, y) = position
    halfSize = size / 2 
    result = []
    result.append(plt.Polygon([(x + halfSize, y), (x - halfSize,y)], edgecolor=color, lw=lineWidth))
    result.append(plt.Polygon([(x, y + halfSize), (x, y - halfSize)], edgecolor=color, lw=lineWidth))
    return result

def CreateCuttingMarks(bigSize, smallSize, markSize, scale = 1):
    (bigW, bigH) = bigSize
    (smallW, smallH) = smallSize
    
    widthOffset = scale * (bigW - smallW) / 2   
    heightOffset = scale * (bigH - smallH) / 2
    
    left = widthOffset
    right = bigW * scale - widthOffset
    up = bigH * scale - heightOffset
    down = heightOffset
    
    result = []
    angles = [(left, down), (left, up), (right, down), (right, up)]
    for a in angles :
        for line in CreateASingleCuttingMark(a, markSize * scale) :
            result.append(line)
            
    return result
    

In [24]:
def ShowCuttingMarks(bleedImg, cutImgSize, scale = 1) :
    fig, ax = plt.subplots()
    markSize = min((bleedImg.size[0] - cutImgSize[0]), (bleedImg.size[1] - cutImgSize[1])) * .8
    for line in CreateCuttingMarks(bleedImg.size, cutImgSize, markSize=markSize, scale=scale) :
        ax.add_patch(line)
    ax.imshow(bleedImg, extent=[0, bleedImg.width * scale, 0, bleedImg.height * scale])
    return fig, ax
    
def ShowCuttingMarksByImage(bleedImg, cutImg, scale = 1) :
    return ShowCuttingMarks(bleedImg, cutImg.size, scale)
    
def ShowCuttingMarksByPath(bleedImgPath, cutImgPath, scale = 1) :
    bleedImg = Image.open(os.path.join(bleedImgPath))
    cutImg = Image.open(os.path.join(cutImgPath))
    return ShowCuttingMarksByImage(bleedImg, cutImg, scale)



def ShowCuttingEdges(bleedImg, cutImgSize, scale = 1) :
    fig, ax = ShowCuttingMarks(bleedImg, cutImgSize, scale)
    ax.add_patch(CreateRectangle(bleedImg.size, cutImgSize, scale))
    return fig, ax
    
def ShowCuttingEdgesByImage(bleedImg, cutImg, scale = 1) :
    return ShowCuttingEdges(bleedImg, cutImg.size, scale)
    
def ShowCuttingEdgesByPath(bleedImgPath, cutImgPath, scale = 1) :
    bleedImg = Image.open(os.path.join(bleedImgPath))
    cutImg = Image.open(os.path.join(cutImgPath))
    return ShowCuttingEdgesByImage(bleedImg, cutImg, scale)


In [8]:
def ShowCuttingEdgesByFolder(folderPath, refBleedImgPath, refCutImgPath, showEdges = False) :
    imageList = os.listdir(folderPath)

    images = [
        Image.open(os.path.join(folderPath, f))
        for f in imageList
    ]

    
    if showEdges :
        ShowCuttingEdgesByPath(refBleedImgPath, refCutImgPath)
    else :
        ShowCuttingMarksByPath(refBleedImgPath, refCutImgPath)
    plt.show()
    
    while (True) :
        npt = input('Prendre cette image comme référence (y/n)?')
        if (npt.strip().lower() == "n") :
            print("NO")
            return
        elif (npt.strip().lower() == "y") :
            print("YES")
            break
    
    cuttingSize = {}
    cuttingSize[Image.open(refBleedImgPath).size] = Image.open(refCutImgPath).size
            
    for img in images :
        display.clear_output(wait=True)
        if (img.size in cuttingSize.keys()) :
            if showEdges :
                ShowCuttingEdges(img, cuttingSize[img.size])
            else :
                ShowCuttingMarks(img, cuttingSize[img.size])
            plt.show()  
            if (input('Continue (y/n)?').strip().lower() == "n") :
                print("NO")
                return
        else :
            print(f"Cutting size ({img.size}) not found")

In [9]:
def computePrintingInfo(pageHeightInCm,pageWidthInCm,
                       maxImageHeightInCm, maxImageWidthInCm,
                       verticalMarginInCm = 0, horizontalMarginInCm = 0,
                       minimalDistanceBetweenImagesInCm = 0) :
    printingInfo = dict()
    printingInfo["pageHeight"] = pageHeightInCm * ptPerCm
    printingInfo["pageWidth"] = pageWidthInCm * ptPerCm
    printingInfo["maxImageHeight"] = maxImageHeightInCm * ptPerCm
    printingInfo["maxImageWidth"] = maxImageWidthInCm * ptPerCm
    printingInfo["verticalMargin"] = verticalMarginInCm * ptPerCm
    printingInfo["horizontalMargin"] = horizontalMarginInCm * ptPerCm
    printingInfo["minimalDistanceBetweenImages"] = minimalDistanceBetweenImagesInCm * ptPerCm
    
    printingInfo["ImagesPerRow"] = (int) ((printingInfo["pageWidth"] - 2 * printingInfo["horizontalMargin"]) / printingInfo["maxImageWidth"])
    #pdfOutput = matplotlib.backends.backend_pdf.PdfPages(r"C:\Users\dodoa\Pictures\MTG\AI Generated Lands\Test\Test.pdf")

In [10]:
class PictureFormat :
    def __init__(self,
                 uncutSizeInPt,
                 cutSizeInPt,
                 cutSizeInUnit,
                 unit = rl.lib.units.cm) :
        self.uncutSizeInPt = uncutSizeInPt
        self.cutSizeInPt = cutSizeInPt
        self.cutSizeInUnit = cutSizeInUnit
        self.unit = unit
        self.resizeFactor = self.GetResizeFactor()
        self.printedUncutSizeInPt = (self.uncutSizeInPt[0] * self.resizeFactor, self.uncutSizeInPt[1] * self.resizeFactor)  
        self.printedCutSizeInPt = (self.cutSizeInPt[0] * self.resizeFactor, self.cutSizeInPt[1] * self.resizeFactor) 
        self.cuttingMarks = dict()
        self.AddCuttingMarks()
    
    def GetResizeFactor(self) :
        verticalSizeFactor = (self.cutSizeInUnit[0] * self.unit) / self.cutSizeInPt[0]
        horizontalSizeFactor = (self.cutSizeInUnit[1] * self.unit) / self.cutSizeInPt[1]
        #print(f"verticalSizeFactor   = {verticalSizeFactor},\nhorizontalSizeFactor = {horizontalSizeFactor}")
        if (abs(verticalSizeFactor - horizontalSizeFactor) < 0.01) :
            sizeFactor = max(verticalSizeFactor, horizontalSizeFactor)
            self.resizeFactor = sizeFactor
            return sizeFactor
        else :
            print ("Factors too far apart...")
            return 0.001
        
    def AddCuttingMarks(self):
        (bigW, bigH) = self.printedUncutSizeInPt
        (smallW, smallH) = self.printedCutSizeInPt
        
        print(f"bigSize = {self.printedUncutSizeInPt}, smallSize = {self.printedCutSizeInPt}")

        widthOffset = (bigW - smallW) / 2   
        heightOffset = (bigH - smallH) / 2

        self.cuttingMarks["left"] = widthOffset
        self.cuttingMarks["right"] = bigW - widthOffset
        self.cuttingMarks["up"] = bigH - heightOffset
        self.cuttingMarks["down"] = heightOffset
    
    def GetUncutSizeInUnit(self) :
        return (self.printedUncutSizeInPt[0] / self.unit, self.printedUncutSizeInPt[1] / self.unit)

    def SetOffset(self, maxImageWidthInPt, maxImageHeightInPt) :
        #print(f"maxImageSizeInPt = {(maxImageWidthInPt, maxImageHeightInPt)}, printedUncutSizeInPt = {self.printedUncutSizeInPt}")
        self.horizontalOffsetInPt = (maxImageWidthInPt - self.printedUncutSizeInPt[0]) / 2
        self.verticalOffsetInPt = (maxImageHeightInPt - self.printedUncutSizeInPt[1]) / 2
    

In [11]:
class PrintingInfo : 
    def __init__(self,
                 pageWidthInUnit, pageHeightInUnit,
                 maxItemWidthInUnit, maxItemHeightInUnit,
                 horizontalMarginInUnit = 0, verticalMarginInUnit = 0,
                 minimalDistanceBetweenItemsInUnit = 0,
                 ptPerUnit = rl.lib.units.cm):
        
        self.pageWidth = pageWidthInUnit * ptPerUnit
        self.pageHeight = pageHeightInUnit * ptPerUnit
        self.maxItemWidth = maxItemWidthInUnit * ptPerUnit
        self.maxItemHeight = maxItemHeightInUnit * ptPerUnit
        self.horizontalMargin = horizontalMarginInUnit * ptPerUnit
        self.verticalMargin = verticalMarginInUnit * ptPerUnit
        self.minimalDistanceBetweenItems = minimalDistanceBetweenItemsInUnit * ptPerUnit
    
        self.itemsPerRow = (int) ((self.pageWidth - 2 * self.horizontalMargin) / self.maxItemWidth)
        self.itemsPerColumn = (int) ((self.pageHeight - 2 * self.verticalMargin) / self.maxItemHeight)

        self.horizontalGap = 0
        self.verticalGap = 0
        if (self.itemsPerRow > 1) :        
            self.horizontalGap = (self.pageWidth - 2 * self.horizontalMargin - self.itemsPerRow * self.maxItemWidth) / (self.itemsPerRow - 1) 
            while (self.horizontalGap < self.minimalDistanceBetweenItems) :
                self.itemsPerRow -= 1
                self.horizontalGap = (self.pageWidth - 2 * self.horizontalMargin - self.itemsPerRow * self.maxItemWidth) / (self.itemsPerRow - 1) 
            
        if (self.itemsPerColumn > 1) :
            self.verticalGap = (self.pageHeight - 2 * self.verticalMargin - self.itemsPerColumn * self.maxItemHeight) / (self.itemsPerColumn - 1) 
            while (self.verticalGap < self.minimalDistanceBetweenItems) :
                self.itemsPerColumn -= 1
                self.verticalGap = (self.pageHeight - 2 * self.verticalMargin - self.itemsPerColumn * self.maxItemHeight) / (self.itemsPerColumn - 1) 

        print(f"Actual layout :\n" + 
              f"{self.itemsPerRow} items per row, {self.itemsPerColumn} items per column ;\n" + 
              f"Horizontal spacing = {(self.horizontalGap / ptPerUnit):.2f} cm ({self.horizontalGap:.2f} pt), " +
              f"Vertical spacing = {(self.verticalGap / ptPerUnit):.2f} cm ({self.verticalGap:.2f} pt).")
              
    
    def itemsPerPage(self) :
        return self.itemsPerRow * self.itemsPerColumn
    
    def GetPositionForItem(self, nItem, isBackside = False) :
        if (nItem >= self.itemsPerPage()) :
            print(f"Cannot print an {n}-th item on a {self.itemsPerRow} x {self.itemsPerColumn} " + 
                  f"grid ({self.itemsPerRow * self.itemsPerColumn} items)")
            return (-1, -1)
        
        if (isBackside) :
            position = ((self.itemsPerRow - 1 - nItem % self.itemsPerRow), (int) (nItem / self.itemsPerRow))
            #print(f"i = {nItem}, position = {position}")
        else :
            position = (nItem % self.itemsPerRow, (int) (nItem / self.itemsPerRow))
            
        resultX = self.horizontalMargin + position[0] * (self.maxItemWidth + self.horizontalGap) 
        resultY = self.verticalMargin + position[1] * (self.maxItemHeight + self.verticalGap)
        
        return (resultX, resultY)
            

In [12]:
def getRealImageSize(uncutSize, cutSize, realCutSizeInCm) :
        realCutSize = (realCutSizeInCm[0] * ptPerCm, realCutSizeInCm[1] * ptPerCm)
        verticalSizeFactor = realCutSize[0] / cutSize[0]
        horizontalSizeFactor = realCutSize[1] / cutSize[1]
        print(f"verticalSizeFactor   = {verticalSizeFactor},\nhorizontalSizeFactor = {horizontalSizeFactor}")
        if (abs(verticalSizeFactor - horizontalSizeFactor) < 0.01) :
            sizeFactor = (verticalSizeFactor + horizontalSizeFactor) / 2
            return (uncutSize[0] * sizeFactor, uncutSize[1] * sizeFactor)
        else :
            print ("Factors too far apart...")
            return (0, 0)

In [18]:
class CanvasManager :
    def __init__(self,
                 pdfResultPath,
                 sourcePicturesFolder,
                 cutPicturesFolder,
                 defaultBacksidePath,
                 defaultImageCutSize,
                 printingPageSize = (29.7, 42),
                 margins = (0, 0),
                 minimalDistanceBetweenImagesInUnit = 0,
                 unit = rl.lib.units.cm,
                 bleedColorRGB = (0, 0, 0)) :
        self.pageMargins = margins
        self.cardMargins = minimalDistanceBetweenImagesInUnit
        self.unit = unit
        self.pageSizeInUnit = printingPageSize
        self.pageSizeInPt = (printingPageSize[0] * unit, printingPageSize[1] * unit)
        self.canvas = rlcanvas.Canvas(pdfResultPath, pagesize= (printingPageSize[0] * self.unit, printingPageSize[1] * self.unit))
        self.pages = 0
        self.itemCount = 0
        self.pdfResultPath = pdfResultPath
        self.sourcePicturesFolder = sourcePicturesFolder
        self.cutPicturesFolder = cutPicturesFolder
        self.defaultBacksidePath = defaultBacksidePath
        self.defaultImageCutSize = defaultImageCutSize
        self.formats = dict()
        self.cuttingMarks = (set(), set())
        self.bleedColorRGB = bleedColorRGB
    
    def GetCorrectCutSize(self, uncutImg, cutImagePaths, showEdges = False) :
        while(True) :
            for cutImgPath in cutImagePaths :
                cutImg = Image.open(cutImgPath)
                
                if (cutImg.size[0] > uncutImg.size[0] and cutImg.size[1] > uncutImg.size[1]) :
                    continue
                
                if showEdges :
                    ShowCuttingEdgesByImage(uncutImg, cutImg)
                else :
                    ShowCuttingMarksByImage(uncutImg, cutImg)
                plt.show()

                npt = input('Use this as layout (y/n)?')
                if (npt.strip().lower() == "y") :
                    #print("YES")
                    return cutImg.size
                display.clear_output(wait=True)
                    
            npt = input('Start over (y/n)?')
            if (npt.strip().lower() == "n") :
                #print("NO")
                return (0, 0)
            
    def InitializeCardBack(self) :
        uncutImg = Image.open(self.defaultBacksidePath)
        self.formats[(0,0)] = PictureFormat(uncutSizeInPt = uncutImg.size,
                                            cutSizeInPt = uncutImg.size,
                                            cutSizeInUnit = self.defaultImageCutSize, 
                                            unit = self.unit)
    
    def GetFormats(self, showEdges = False) :
        uncutImgNames = os.listdir(self.sourcePicturesFolder)
        uncutImagePaths = [
            os.path.join(self.sourcePicturesFolder, f)
            for f in os.listdir(self.sourcePicturesFolder)#uncutImgNames
        ]        
        #cutImagNames = os.listdir(self.cutPicturesFolder)
        cutImagePaths = [
            os.path.join(self.cutPicturesFolder, f)
            for f in os.listdir(self.cutPicturesFolder)#cutImagNames
        ]

        for uncutImgPath in uncutImagePaths :
            uncutImg = Image.open(uncutImgPath)
            
            if (uncutImg.size in self.formats.keys()):
                continue 
            
            cutImgSize = self.GetCorrectCutSize(uncutImg, cutImagePaths, showEdges)
            self.formats[uncutImg.size] = PictureFormat(uncutSizeInPt = uncutImg.size,
                                                        cutSizeInPt = cutImgSize,
                                                        cutSizeInUnit = self.defaultImageCutSize, 
                                                        unit = self.unit)
        self.InitializeCardBack()
            
    def CreatePrintingInfo(self) :
        maxImageWidthInUnit = 0
        maxImageHeightInUnit = 0
        maxImageWidthInPt = 0
        maxImageHeightInPt = 0
        for f in self.formats.values() :
            (w, h) = f.GetUncutSizeInUnit()
            #print(f"w= {w}, h= {h}")
            if (maxImageWidthInPt < f.printedUncutSizeInPt[0]):
                maxImageWidthInUnit = w
                maxImageWidthInPt = f.printedUncutSizeInPt[0]
                
            if (maxImageHeightInPt < f.printedUncutSizeInPt[1]) :
                maxImageHeightInUnit = h
                maxImageHeightInPt = f.printedUncutSizeInPt[1]
            
        self.maxImageWidthInPt = maxImageWidthInPt
        self.maxImageHeightInPt = maxImageHeightInPt
            
        for f in self.formats.values() :
            f.SetOffset(maxImageWidthInPt, maxImageHeightInPt)
            
        #print(f"pageWidthInUnit = {self.pageSizeInUnit[0]},  pageHeightInUnit = {self.pageSizeInUnit[1]},  maxItemWidthInUnit = {maxImageWidthInUnit},  maxItemHeightInUnit = {maxImageHeightInUnit},  horizontalMarginInUnit = {self.pageMargins[0]},  verticalMarginInUnit = {self.pageMargins[1]},  minimalDistanceBetweenItemsInUnit = {self.cardMargins},  ptPerUnit = {self.unit}")
        self.printingInfo = PrintingInfo(pageWidthInUnit = self.pageSizeInUnit[0], 
                                         pageHeightInUnit = self.pageSizeInUnit[1],
                                         maxItemWidthInUnit = maxImageWidthInUnit, 
                                         maxItemHeightInUnit = maxImageHeightInUnit,
                                         horizontalMarginInUnit = self.pageMargins[0], 
                                         verticalMarginInUnit = self.pageMargins[1],
                                         minimalDistanceBetweenItemsInUnit = self.cardMargins,
                                         ptPerUnit = self.unit)
    
    def DrawCuttingMarks(self) :
        markLength = .9 * self.printingInfo.horizontalMargin
        for horizontalMark in self.cuttingMarks[0] :
            self.canvas.line(0, horizontalMark, markLength, horizontalMark)
            self.canvas.line(self.pageSizeInPt[0], horizontalMark, self.pageSizeInPt[0] - markLength, horizontalMark)
        
        markLength = .9 * self.printingInfo.verticalMargin
        for verticalMark in self.cuttingMarks[1] :
            self.canvas.line(verticalMark, 0, verticalMark, markLength)
            self.canvas.line(verticalMark, self.pageSizeInPt[1], verticalMark, self.pageSizeInPt[1] - markLength)
            
        
    def AddPage(self) :
        self.DrawCuttingMarks()
        self.canvas.showPage()
        self.cuttingMarks = (set(), set())
        
        if (self.itemCount % self.printingInfo.itemsPerPage() == 0) :
            iterations = self.printingInfo.itemsPerPage()
        else :
            iterations = self.itemCount % self.printingInfo.itemsPerPage()
        #print(f"iterations = {iterations}")
            
        for i in range (iterations) :
            self.AddItem(self.defaultBacksidePath, bleed = True, isBackside = True, forceIndex = i)
            
        self.DrawCuttingMarks()
        self.canvas.showPage()
        self.cuttingMarks = (set(), set())
        self.pages += 1

    def AddItem(self, itemPath, bleed = False, isBackside = False, forceIndex = 0) :
        uncutImg = Image.open(itemPath)
        if (isBackside) :
            imgFormat = self.formats[(0, 0)]
        else :
            imgFormat = self.formats[uncutImg.size]
        
        if (isBackside == False and (self.itemCount > 0 and (self.itemCount % self.printingInfo.itemsPerPage() == 0))) :
            self.AddPage()
        indexOnPage = self.itemCount % self.printingInfo.itemsPerPage()
        
        if (isBackside == False) :
            forceIndex = indexOnPage
        position = self.printingInfo.GetPositionForItem(forceIndex, isBackside)
        
        itemName = {itemPath[itemPath.rindex('\\'):itemPath.rindex('.')]}
        #print(f"position for item #{indexOnPage} ({itemName}) : {position}")
        
        if (bleed) :
            self.canvas.setFillColorRGB(self.bleedColorRGB[0], self.bleedColorRGB[1], self.bleedColorRGB[2])
            self.canvas.rect(position[0], position[1], self.maxImageWidthInPt, self.maxImageHeightInPt, stroke = 0, fill = 1)
        
        position = (position[0] + imgFormat.horizontalOffsetInPt, position[1] + imgFormat.verticalOffsetInPt)
        #print(f"position for item #{indexOnPage} ({itemName}) : {position}")
        
        self.cuttingMarks[0].update({position[1] + imgFormat.cuttingMarks["up"], position[1] + imgFormat.cuttingMarks["down"]})
        self.cuttingMarks[1].update({position[0] + imgFormat.cuttingMarks["left"], position[0] + imgFormat.cuttingMarks["right"]})
        
        self.canvas.drawImage(itemPath,
                              position[0], 
                              position[1],
                              width = imgFormat.printedUncutSizeInPt[0],
                              height = imgFormat.printedUncutSizeInPt[1])
        
        if (not isBackside) :
            self.itemCount += 1
        
    def PrintEveryItem(self, showEdges = True, bleed = False) :
        mngr.GetFormats(showEdges)
        display.clear_output(wait=True)
        
        self.CreatePrintingInfo()
        if (input("Continue with this layout (y/n) ?").lower() == "n") :
            return
        
        self.cuttingMarks = (set(), set())
        
        uncutImagePaths = [
            os.path.join(self.sourcePicturesFolder, f)
            for f in os.listdir(self.sourcePicturesFolder)
        ]
        
        for img in uncutImagePaths :
            self.AddItem(img, bleed = bleed) 
            display.clear_output(wait=True)
            print(f"Progression : {self.itemCount}/{len(uncutImagePaths)} done...")
        
        self.AddPage()
        self.canvas.save()
        
        

In [ ]:
mngr = CanvasManager(pdfResultPath = r'C:\Users\dodoa\Pictures\MTG\Proxies\EDH Lands MPC Ready\PrintSheet.pdf',
                 sourcePicturesFolder = r'C:\Users\dodoa\Pictures\MTG\Proxies\EDH Lands MPC Ready\To Print',
                 cutPicturesFolder = r'C:\Users\dodoa\Pictures\MTG\Proxies\EDH Lands MPC Ready\Cropped',
                 defaultBacksidePath = r'C:\Users\dodoa\Pictures\MTG\Card Backs\Card Back V5.png',
                 defaultImageCutSize = (5.9, 8.6),#(6.35, 8.89),
                 printingPageSize = (42, 29.7),
                 margins = (.05, .5),
                 minimalDistanceBetweenImagesInUnit = 0,
                 unit = rl.lib.units.cm)
mngr.PrintEveryItem()

Actual layout :
6 items per row, 3 items per column ;
Horizontal spacing = 0.29 cm (8.31 pt), Vertical spacing = 0.55 cm (15.66 pt).
